# Terrestrial Hydrology Fall 2023 Lab 6 - Pumping drawdown modeling
Hi everyone! This is our sixth and final  python lab assignment. Just as last time, you will complete these assignments with a mix of responses as code and narrative text within this jupyter notebook document. To complete the assignment, go to File > Print Preview, and export as a PDF (.pdf file) before uploading the assignment to Gradescope.

Look for triple @@@ signs in places where we expect you to write code or respond to a question. The rest of the notebook should not require modification.

This assignment is based on code developed by Walt McNab, who is a computational environmental scientist who has developed many scripts to demonstrate common concepts in hydrogeology. You can read more about him and find other tools/examples here: https://numericalenvironmental.wordpress.com/

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib import dates
import matplotlib.dates as mdates
import scipy
from scipy.special import *
from scipy.spatial import distance
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

# [1] Defining Functions
Walt McNab's code is a very nice example of Object-Oriented Programming (OOP). Read more on this method here: https://realpython.com/python3-object-oriented-programming/. Here, we will use his code to define a number of functions for use in this lab. 

In [ ]:
#The Aquifer class is simply a container for aquifer properties (hydraulic conductivity, specific storage, and thickness) which are all assumed to be uniform

class Aquifer: # container for aquifer properties (uniform)

    def __init__(self, K, Ss, b):
        self.K = K                  # hydraulic conductivity
        self.Ss = Ss                # specific storage
        self.b = b                  # aquifer thickness


In [ ]:
class Well: #The Well class contains well characteristics (location, pumping rate)

    def __init__(self, x, y, Q, aquifer):
        self.loc = np.array([[x, y]])       # well coordinates
        self.Q = Q                          # pumping rate
        self.aquifer = aquifer              # aquife robject

    def W(self, u): # A Well object is associated with each pumping well in the model
        # exponential integral/well function
        return expn(1,u)

    def S(self, r, t):
        # change in hydraulic head h0 - h(r,t) from pumping
        u = (r**2)*self.aquifer.Ss*self.aquifer.b/(4.*self.aquifer.K*self.aquifer.b*t) 
        return self.Q*self.W(u)/(4.*np.pi*self.aquifer.K*self.aquifer.b)

    def Impact(self, pts, t):
        # calculate the hydraulic head impact from well to a 
        # n x 2 vector of (x, y) points, loc
        r = distance.cdist(self.loc, pts)[0]
        return self.S(r, t)

In [ ]:
#The Grid class contains an array of points. Each point is characterized by a total drawdown, with a contribution from each well in the model. 
#A broadcasting approach is used to speed up calculations, so that a large number of both grid points and wells can be used, if warranted in the model:

class Grid:

    def __init__(self, x0, y0, xf, yf, num_x, num_y):
        # create a uniform grid with supplied specs and 
        # convert to a n x 2 array
        x = np.linspace(x0, xf, num_x+1)
        y = np.linspace(y0, yf, num_y+1)
        X, Y = np.meshgrid(x,y)
        self.points = np.matrix([X.flatten(), Y.flatten()]).T
        self.S = np.zeros(len(self.points), float)

    def Calculate(self, well, t):
        
        # calculate net drawdown and write to output file
        for w in well:
            self.S += w.Impact(self.points, t)    
        final_head = np.empty((3,int(len(self.points))))
        for (i, pt) in enumerate(self.points):
            final_head[0,i] = pt[0, 0]
            final_head[1,i] = pt[0, 1]
            final_head[2,i] = self.S[i]
        return final_head
        

In [ ]:
# The following function simply reads the input file for the grid (domain) definition and creates corresponding objects:
def ReadParams():
    # read model parameters and assign to appropriate object
    param = []     
    input_file = open('params.txt','r')
    for line in input_file:
        line_input = line.split()
        param.append(line_input[1]) 
    input_file.close()
           # aquifer thickness
    x0 = float(param[0])        # grid margins, x-direction
    xf = float(param[1])
    y0 = float(param[2])        # grid margings, y-direction
    yf = float(param[3])
    num_x = int(param[4])       # grid discretization
    num_y = int(param[5])
    grid = Grid(x0, y0, xf, yf, num_x, num_y)
    return grid
    

In [ ]:
# The following function takes in the well position/Q information and creates corresponding objects:
def ReadWells(aquifer,well_loc):
    # read well matrix and assign well objects
    well = []
    input_file = well_loc
    if len(input_file.shape)>1:
        for i in range(input_file.shape[0]):
            line_input = input_file[i,:]
            well.append(Well(float(line_input[0]), float(line_input[1]),
                float(line_input[2]), aquifer))
    else:
        well.append(Well(float(line_input[0]), float(line_input[1]),
                float(line_input[2]), aquifer))
    return well            

# Question 1

Within the last 5 cells, identify the place where the actual drawdown calculation happens. Comparing the equation to the lecture notes, what specific scenario are we solving for here? What is this equation called?

@@@ 

# [2] Choose input parameters for your aquifer & plot
Set the hydraulic conductivity K to 40 m/day, the storativity to 2.50*10^(-4) and the thickness to 30 m.

In [ ]:
K = @@@     # hydraulic conductivity
Ss = @@@     # storativity
b = @@@ # aquifer thickness

The domain is a square of 25000 by 25000 m.  **Here we are placing a single well at the center of the domain with a pumping rate of -10^5 m^3/day. The array takes x-location, y-location, and flow rate, in this order.** We will use this syntax later, so make sure to look at this! 

In [ ]:
well_loc = np.array([[12500, 12500, -10**5]])

The following box now reads the parameters and sets up the well location in the grid. You will have to rerun the below code when you change the aquifer parameters. 

In [ ]:
grid = ReadParams()
well_input = ReadWells(Aquifer(K, Ss, b), well_loc)

Below is an example on how to generate  3d and 2d plots for t = 50 days. Note you might get an error running the below code block, but two plots should be produced (the first 3D the second 2D). 

In [ ]:
t = 50
hh = grid.Calculate(well_input, t)
fig, ax = plt.subplots(subplot_kw={'projection': '3d'})
ax.plot_trisurf(hh[0], hh[1], hh[2], cmap = cm.viridis)
ax.set_title('t = 50 days');
plt.show()


## The following code will show you a birds-eye view of the same plot:
# we put the data into a dataframe
df = pd.DataFrame(dict(x=hh[0], y=hh[1], z=hh[2]))
xcol, ycol, zcol = "x", "y", "z"
#sort values
df = df.sort_values(by=[xcol, ycol])
xvals = df[xcol].unique()
yvals = df[ycol].unique()
#reshape the drawdown values into a 2D map
zvals = df[zcol].values.reshape(len(xvals), len(yvals)).T

# Now do the actual plotting
cs = plt.contourf(xvals, yvals, zvals)
plt.colorbar()
plt.contour(cs, colors='k')
plt.show()

# [3] Modifying the Well Space

# Question 2
How does the drawdown evolve between day 50, day 100, and day 200? Plot the necessary figures to answer this question.

@@@

In [ ]:
@@@

# Question 3
How does the drawdown evolve when the conductivity is twice the intial one? What about when it is half? Does this make sense? Plot the necessary figures to answer this question.

@@@

In [ ]:
@@@

# Question 4
How does the drawdown evolve when the aquifer thickness is twice the intial one? What about when it is half? Plot the necessary figures to answer this question.

@@@ 

In [ ]:
@@@

# Question 5
Place two wells with equal pumping rates far away from each other using the syntax: well_loc = np.array([[xloc1, yloc1, Q1],[xloc2, yloc2, Q2]]). Vary the pumping rate (Q values), conductivity and aquifer thickness to see how various values will affect the entire domain. 

Then, show a figure of an interesting pattern you obtained. 

And answer: how do you need to modify the pumping rate and/or aquifer parameters to see a drawdown in between the two wells? 

@@@

In [ ]:
@@@

# Question 6
Place five or more wells in the domain. Play with well placements and pumping rate to develop intuition for how the various wells will interact with each other. Show a figure of an interesting pattern you obtained.

In [ ]:
@@@

# THE END! :) 